In [58]:
"""
使用Xgboost集成学习方法
"""
import numpy as np
from keras.datasets import fashion_mnist
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import xgboost as xgb

sp = 12000
spp = 2000
# sp = 24000
# spp = 4000
# n_size = 28 * 28

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X = np.float32(np.concatenate((X_train[:sp], X_test[:spp]), axis=0).reshape(sp+spp, n_size))
y = np.float32(np.concatenate((y_train[:sp], y_test[:spp]), axis=0))

print("X shape:", X.shape, "\ny shape:",y.shape)

pca = PCA(n_components=0.85)
X = pca.fit_transform(X)
print("PCA Finish! {} features selected and explained_variance_ratio:\n{}\n".format(len(pca.explained_variance_ratio_), pca.explained_variance_ratio_))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20)

print("X train shape:", X_train.shape, "y train shape:", y_train.shape)
print("X valid shape:", X_val.shape, "y valid shape:", y_val.shape)
print("X test shape:", X_test.shape, "y test shape:", y_test.shape)

num_round = 300
xgt_param = {}
xgt_param['objective'] = 'multi:softmax'  

xgt_param['num_class'] = 10
xgt_param['eta'] = 0.1
xgt_param['max_depth'] = 6
xgt_param['subsample'] = 0.8
xgt_param['colsample_bytree'] = 0.8
xgt_param['alpha'] = 1
xgt_param['gamma'] = 0
xgt_param['eval_metric']="merror"
xgt_param['scale_pos_weight'] = 1
xgt_param['seed'] = 225


xg_train = xgb.DMatrix(X_train, label=y_train)
xg_val = xgb.DMatrix(X_val, label=y_val)
xg_test = xgb.DMatrix(X_test, label=y_test)

watchlist = [(xg_train, 'train'), (xg_val, 'val')]
bst = xgb.train(xgt_param, xg_train, num_round, watchlist)
y_pred = bst.predict(xg_test)
error_rate = np.sum(y_pred != y_test) / y_test.shape
print('Test error using softmax = {} ({})'.format(error_rate, 1-error_rate))

# 2018/5/31 Default xgboost Acc Test: 0.8348 (with PCA)
# 2018/5/31 xgboost validation_0, merror:0.133036, Acc Test: 0.8617(with PCA 0.90)
# 2018/5/31 validation_0, merror:0.11875, Acc Test: 0.87607; (without PCA or Scaling)
# 2018/5/31 xgboost bad 0.4(without PCA, with Scaling)

# 2018//6/2 xgboost Test error using softmax = [0.12785714] ([0.87214286])
"""
pca = PCA(n_components=0.85)

num_round = 500
xgt_param = {}
xgt_param['objective'] = 'multi:softmax'  
xgt_param['num_class'] = 10
xgt_param['eta'] = 0.1
xgt_param['max_depth'] = 6
xgt_param['subsample'] = 0.8
xgt_param['colsample_bytree'] = 0.8
xgt_param['alpha'] = 1
xgt_param['gamma'] = 0
xgt_param['eval_metric']="merror"
xgt_param['scale_pos_weight'] = 1
"""


X shape: (14000, 784) 
y shape: (14000,)
PCA Finish! 43 features selected and explained_variance_ratio:
[0.29202336 0.17851242 0.05974947 0.04919249 0.0383718  0.033541
 0.02334837 0.01908923 0.01347624 0.01338953 0.0099831  0.00891331
 0.00767028 0.00657983 0.0061076  0.00606144 0.0054462  0.00527842
 0.00458113 0.00452859 0.00440668 0.00410413 0.00386382 0.00373824
 0.00362934 0.00348898 0.00331655 0.00319408 0.00310875 0.00290465
 0.00276278 0.00264419 0.00262288 0.0025419  0.00248121 0.00234792
 0.00230013 0.00225029 0.00220079 0.00206569 0.00198936 0.00197893
 0.00195777]

X train shape: (8960, 43) y train shape: (8960,)
X valid shape: (2240, 43) y valid shape: (2240,)
X test shape: (2800, 43) y test shape: (2800,)
[0]	train-merror:0.180134	val-merror:0.248661
[1]	train-merror:0.152009	val-merror:0.21875
[2]	train-merror:0.139509	val-merror:0.205804
[3]	train-merror:0.13404	val-merror:0.194643
[4]	train-merror:0.128683	val-merror:0.195982
[5]	train-merror:0.121094	val-merror:0.192

'\npca = PCA(n_components=0.85)\n\nnum_round = 500\nxgt_param = {}\nxgt_param[\'objective\'] = \'multi:softmax\'  \nxgt_param[\'num_class\'] = 10\nxgt_param[\'eta\'] = 0.1\nxgt_param[\'max_depth\'] = 6\nxgt_param[\'subsample\'] = 0.8\nxgt_param[\'colsample_bytree\'] = 0.8\nxgt_param[\'alpha\'] = 1\nxgt_param[\'gamma\'] = 0\nxgt_param[\'eval_metric\']="merror"\nxgt_param[\'scale_pos_weight\'] = 1\n'

In [41]:
xgt_param['objective'] = 'multi:softprob'
bst = xgb.train(xgt_param, xg_train, num_round, watchlist)
y_pred = bst.predict(xg_test)
pred_prob = bst.predict(xg_test).reshape(y_test.shape[0], 10)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {} ({})'.format(error_rate, 1-error_rate))

[0]	train-merror:0.204632	val-merror:0.244643
[1]	train-merror:0.175167	val-merror:0.225446
[2]	train-merror:0.163114	val-merror:0.213839
[3]	train-merror:0.15385	val-merror:0.207589
[4]	train-merror:0.146875	val-merror:0.202232
[5]	train-merror:0.141183	val-merror:0.20067
[6]	train-merror:0.139732	val-merror:0.19442
[7]	train-merror:0.137109	val-merror:0.192188
[8]	train-merror:0.132087	val-merror:0.189509
[9]	train-merror:0.128237	val-merror:0.189732
[10]	train-merror:0.126507	val-merror:0.1875
[11]	train-merror:0.124609	val-merror:0.185268
[12]	train-merror:0.120368	val-merror:0.184375
[13]	train-merror:0.118248	val-merror:0.184598
[14]	train-merror:0.113728	val-merror:0.183259
[15]	train-merror:0.111551	val-merror:0.181027
[16]	train-merror:0.108371	val-merror:0.180134
[17]	train-merror:0.105804	val-merror:0.179018
[18]	train-merror:0.103069	val-merror:0.174777
[19]	train-merror:0.099498	val-merror:0.17433
[20]	train-merror:0.096261	val-merror:0.174777
[21]	train-merror:0.093248	va